In [1]:
import pandas as pd
import numpy as np
import nltk
import seaborn as sns

In [2]:
columns=['target','ids','date','flag','user','text']

In [3]:
doc=pd.read_csv("twitter_api.csv",names=columns)

In [4]:
doc.shape

(1600000, 6)

In [5]:
doc.head(5)

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
doc_t=doc[['target','text']]

In [7]:
doc_t.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [8]:
doc_t['target']=doc['target'].map({0:0,4:1})

<ipython-input-8-90ba07c872e4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_t['target']=doc['target'].map({0:0,4:1})


In [9]:
doc_t['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

In [10]:
data_pos=doc_t[doc_t['target']==0]
data_neg=doc_t[doc_t['target']==1]

In [11]:
data_p=data_pos[0:1000]

In [12]:
data_p.shape

(1000, 2)

In [13]:
data_n=data_neg[0:1000]

In [14]:
data_pos.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [15]:
dataset=pd.concat([data_p,data_n])

In [16]:
dataset.shape

(2000, 2)

In [33]:
data_train=dataset[0:1500]
data_test=dataset[1500:]

In [18]:
data_train.head(1)

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."


In [19]:
doc=np.array(data_train)

In [34]:
doc_test=np.array(data_test)

In [20]:
from nltk.corpus import stopwords
import string
stops=stopwords.words('english')+list(string.punctuation)

In [29]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.tokenize import word_tokenize

In [22]:
pos_tag(["hi all", 'good'])

[('hi all', 'NN'), ('good', 'JJ')]

In [23]:
lem=WordNetLemmatizer()

In [24]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [25]:
def NotNumber(w):
    for ch in w:
        if(ch>='0' and ch<='9'):
            return False
    return True

In [26]:
def clean_review(words):
    doc=word_tokenize(words)
    doc=[word for word in doc if word.lower() not in stops and NotNumber(word)==True]
    pos=pos_tag(doc)
    cleaned_words=[lem.lemmatize(w,pos=get_simple_pos(t)) for w,t in pos]
    string=""
    for w in cleaned_words:
        string+=w
        string+=' '
    return string

In [30]:
data_train1=[(clean_review(doc_w[1]),doc_w[0]) for doc_w in doc ]

In [35]:
data_test1=[(clean_review(w_test[1])) for w_test in doc_test]

In [31]:
import random
random.shuffle(data_train1)

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

In [36]:
count_vec=CountVectorizer(max_features=3000,ngram_range=(1,2),max_df=0.3)
count_vec.fit([doc[0] for doc in data_train1])

CountVectorizer(max_df=0.3, max_features=3000, ngram_range=(1, 2))

In [37]:
train_x = count_vec.transform([doc[0] for doc in data_train1])
train_y=[doc[1] for doc in data_train1]

In [38]:
test_x=count_vec.transform(data_test1)

In [39]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.8)
clf.fit(train_x,train_y)

MultinomialNB(alpha=0.8)

In [40]:
test_y=clf.predict(test_x)
clf.score(train_x,train_y)

0.9206666666666666